In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00


In [ ]:
!apt-get install git
!git clone https://github.com/ryanzhumich/AESLC.git
import os
# Path to the 'enron_subject_line' folder within 'AESLC'
folder_path = '/content/AESLC/enron_subject_line'
# List contents of the 'enron_subject_line' folder
contents = os.listdir(folder_path)
print(contents)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 5.50 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Updating files: 100% (18304/18304), done.
['dev', 'test', 'train']


In [ ]:
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"  #"gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def process_subject_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        subject_text = file.read().strip()
        return {'File': os.path.basename(file_path), 'Subject': subject_text}

def create_csv(directory_path, output_csv):
    data = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.subject'):
            file_path = os.path.join(directory_path, filename)
            data.append(process_subject_file(file_path))

    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)

input_directory = "/content/AESLC/enron_subject_line/train"
output_csv_file = "output.csv"
create_csv(input_directory, output_csv_file)

In [ ]:

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/output.csv')

# Assuming your column is named 'text_and_subject'
df['Text'] = df['Subject'].apply(lambda x: x.split('@subject')[0].strip())
df['Sub'] = df['Subject'].apply(lambda x: x.split('@subject')[1].strip() if '@subject' in x else '')

# Drop the original column if needed
df.drop(columns=['Subject','File'], inplace=True)



In [ ]:
df.head(5)

,Text,Sub
0,Could you please ensure that these books are s...,Books
1,Tana - Dan Diamond called me.\nHe needs new d...,TFS Energy LLC
2,This is an excerpt from an internal Weather Ch...,FYI - Weather Channel Ratings
3,I just wanted to clarify the purpose and file ...,Location of Curve Validation Progress Tracking...
4,Frank: BEWARE!\nThis ISDA was almost finalize...,NYSEG


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df.iterrows():
    input_text = row['Text']
    subject = row['Sub']
    input_text = " ".join(input_text.split()[:300])
    out = input_text + " \n@subject\n " + subject + " \n<EMAIL_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/gpt_model',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,3.388100
1000,3.293200
1500,3.189900
2000,3.086000
2500,3.099000
3000,3.040100
3500,3.022400
4000,2.860100
4500,2.844500
5000,2.820700


TrainOutput(global_step=10827, training_loss=2.8783402527932194, metrics={'train_runtime': 3876.2679, 'train_samples_per_second': 11.173, 'train_steps_per_second': 2.793, 'total_flos': 5943536363520000.0, 'train_loss': 2.8783402527932194, 'epoch': 3.0})

In [ ]:
model_output_path = "/content/model"

In [ ]:
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json')

In [ ]:
def generate_response(model, tokenizer, prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Load the fine-tuned model and tokenizer
my_model = GPT2LMHeadModel.from_pretrained(model_output_path)
my_tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [ ]:
prompt = "The 2002 Enron Net Works budget will for the first time combine allocations to business units for four key services:  Enron Online, IT Infrastructure, Risk Management Operations and IT Maintenance and Development. We are providing a level of detail for expense and capital spend that is greater than that provided in past years, and that will allow business units to make informed choices regarding the services that they receive.We are meeting with individual business unit Offices of the Chair to review in detail the four service areas delivered through Enron Net Works.Some of these meetings have been completed and more will occur between now and October 12.Please keep the following points in mind as we work with you to finalize budget:  Enron Networks today is very different from this tim"  # Input prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)    #Actual Subject: Enron Net Works 2002 Budget Process

In [ ]:
import os
import csv
import re
from tqdm import tqdm  # Import tqdm for progress bar


# Folder path containing the files
folder_path = "/content/AESLC/enron_subject_line/dev"

# Output CSV file path
csv_output_path = "model_dev_output.csv"

# Model saving path
model_save_path = "/content/model"
os.makedirs(model_save_path, exist_ok=True)

# Initialize an empty list to store data
data = []

# Iterate through files in the folder
for iteration, filename in enumerate(tqdm(os.listdir(folder_path), desc="Processing files")):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r') as file:
          try:
            content = file.read()

            # Extracting prompt and annotations using string splitting
            prompt = content.split('@subject')[0].strip()[:800] + " @subject" if '@subject' in content else ''
            print(f"\nIteration: {iteration}")
            # print(f"Modified Prompt: {prompt}")

            subject = content.split('@subject')[1].split('@')[0].strip() if '@subject' in content else ''

            ann0 = content.split('@ann0')[1].split('@')[0].strip() if '@ann0' in content else ''
            # print(f"Annotation 0: {ann0}")

            ann1 = content.split('@ann1')[1].split('@')[0].strip() if '@ann1' in content else ''
            # print(f"Annotation 1: {ann1}")

            ann2 = content.split('@ann2')[1].split('@')[0].strip() if '@ann2' in content else ''
            # print(f"Annotation 2: {ann2}")

            # Generating a response (replace with your actual model inference logic)
            response = generate_response(my_model, my_tokenizer, prompt)
            # print(f"Generated Response: {response}")

            # Append data to the list
            data.append([prompt, subject, ann0, ann1, ann2, response])
          except Exception as e:
                print(f"Error reading file {filename}: {e}")

# Write data to CSV file
with open(csv_output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

    # Write data rows
    csvwriter.writerows(data)

print(f"CSV file '{csv_output_path}' created successfully.")


In [ ]:
import os
import csv
from tqdm import tqdm  # Import tqdm for progress bar


# Folder path containing the files
folder_path = "/content/AESLC/enron_subject_line/test"

# Output CSV file path
csv_output_path = "model_test_output.csv"

# Model saving path
model_save_path = "/content/model"
os.makedirs(model_save_path, exist_ok=True)

# Initialize an empty list to store data
data = []

# Number of iterations to save the model
save_model_interval = 5

# Iterate through files in the folder
for iteration, filename in enumerate(tqdm(os.listdir(folder_path), desc="Processing files")):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r') as file:
          try:
            content = file.read()

            # Extracting prompt and annotations using string splitting
            prompt = content.split('@subject')[0].strip()[:800] + " @subject" if '@subject' in content else ''
            print(f"\nIteration: {iteration}")
            # print(f"Modified Prompt: {prompt}")

            subject = content.split('@subject')[1].split('@')[0].strip() if '@subject' in content else ''

            ann0 = content.split('@ann0')[1].split('@')[0].strip() if '@ann0' in content else ''
            # print(f"Annotation 0: {ann0}")

            ann1 = content.split('@ann1')[1].split('@')[0].strip() if '@ann1' in content else ''
            # print(f"Annotation 1: {ann1}")

            ann2 = content.split('@ann2')[1].split('@')[0].strip() if '@ann2' in content else ''
            # print(f"Annotation 2: {ann2}")

            # Generating a response (replace with your actual model inference logic)
            response = generate_response(my_model, my_tokenizer, prompt)
            # print(f"Generated Response: {response}")

            # Append data to the list
            data.append([prompt, subject, ann0, ann1, ann2, response])
          except Exception as e:
                print(f"Error reading file {filename}: {e}")

# Write data to CSV file
with open(csv_output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

    # Write data rows
    csvwriter.writerows(data)

print(f"CSV file '{csv_output_path}' created successfully.")


In [ ]:
my_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import pandas as pd
import csv


In [ ]:
data_dev = pd.read_csv("/content/model_dev_output.csv")

In [ ]:
data_test = pd.read_csv("/content/model_test_output.csv")

In [ ]:
data_dev.head(5)

,Prompt,Subject,Annotation 0,Annotation 1,Annotation 2,Generated Subject
0,"Dear Marketer, In an effort to ensure proper ...",EBB Operations Notification,need update on contact information,contact information needed,update contact info,"Dear Marketer, In an effort to ensure proper ..."
1,Welcome back all of you veteran pickers!\nI?d ...,PICKS OF THE WEEK - WEEK 1,introducing our three new rookies,new pickers introduced for the new contest wee...,football pool picks,Welcome back all of you veteran pickers!\nI?d ...
2,"Guys, I will set up a monthly meeting between ...",ENA Staff Meetings,will be setting up monthly meeting,monthly meeting,"deal flow, priorities, and strategy meeting","Guys, I will set up a monthly meeting between ..."
3,"Dear Alumni, Please see the career opportunit...",Career Opportunity: Director of Consulting wit...,check out our new non-profit for opportunities,career opportunity for director,director of consulting job opening,"Dear Alumni, Please see the career opportunit..."
4,The 2002 Enron Net Works budget will for the f...,Enron Net Works 2002 Budget Process,enron net works budgets,enron net works,budget for allocations,The 2002 Enron Net Works budget will for the f...


In [ ]:
def extract_subject(subject_text):
    start = subject_text.find('@subject') + len('@subject')
    end = subject_text.find('@subject', start)
    if end != -1:
        return subject_text[start:end].strip()
    else:
        return subject_text[start:].strip()

In [ ]:
def get_subject(subject_text):
  extracted_text = extract_subject(subject_text).split('<EMAIL_ENDED>', 1)[0].strip()
  return extracted_text

In [ ]:
data_dev['Generated Subject'] = data_dev['Generated Subject'].apply(get_subject)

In [ ]:
data_test['Generated Subject'] = data_test['Generated Subject'].apply(get_subject)

In [ ]:
data_test.head(10)

,Prompt,Subject,Annotation 0,Annotation 1,Annotation 2,Generated Subject
0,It is that time again for the mandatory Annual...,Annual Compliance Meeting,annual compliance meeting agenda,annual compliance meeting schedule details,mandatory annual compliance meeting,.
1,"I'm attaching the final copy of my resume, alr...",Third time's a charm,attaching final copy of my resume,final copy of my resume,please review:attached completed resume.,Resume
2,The Pulp Weekly Meeting scheduled by Rodney Ma...,Pulp Weekly Meeting - ROOM CHANGED,meeting room changes,pulp weekly meeting,pulp weekly meeting location change,"Pulp Weekly Meeting - Monday, September 28, 2001"
3,"All, We will be scheduling the next recruitme...",Recruitment - Trading Track,may's recruitment day,recruitment day,recruitment day schedule,ENA Trading Track
4,"Susan, Under the terms of the Gallup Compre...",Gallup Compressor Station,tri-state confidentiality agreement to review,tri state confidentiality agreement,my thoughts on the tri-state agreement,Gallup Compression Services Agreement
5,"FYI, The next Outage Coordination Commercial ...",Outage Coordination Commercial Review Meeting ...,outage coordination meeting today 03/12/02,outage coordination commercial review meeting,outage coordination commercial review meeting ...,Outage Coordination Commercial Review Meeting
6,I don't think this is very usefull but you guy...,CNG MARQ Space from the pipeline,marq capacity available,marq capacity bids,bids being accepted for marq,CNG
7,"""It's a Girl"" performed by Weave Dance Company...",Enron In Action 05.21.01,it's a girl to star wars : upcoming events for...,enron employees upcoming events,time listings for performances,Weave Dance Company
8,Greetings Catherine and Roger: A PUC staffer ...,PUC Inquiring About Enron Turnback of CA Gas C...,issue with information,regarding call from puc staffer,turning back ontario school?,= Direct Access )
9,I am coming in this weekend to box up for the ...,Clean Sweep,instructions for big move,the big move,you want it moved? follow these instructions!,Move Packet


In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=44c010a47ff9e03be01b5335630f382cf527b3e3708f9eccbbdd153d43514906
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from evaluate import load
from collections import defaultdict
from tqdm import tqdm

def metric(data):
    sub,ann0,ann1,ann2,gen_subject  = ["Subject","Annotation 0","Annotation 1","Annotation 2","Generated Subject"]
    rouge = load("rouge")
    meteor = load("meteor")
    score = defaultdict(int)

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_s = rouge.compute(predictions=[x[gen_subject]],references=[x[sub]])
        rouge_h = rouge.compute(predictions=[x[gen_subject]] ,references=[[x[ann0],x[ann1],x[ann2]]])

        score["rouge_1_sub"] += rouge_s['rouge1']
        score["rouge_2_sub"] += rouge_s['rouge2']
        score["rouge_L_sub"] += rouge_s['rougeL']

        score["rouge_1_human"] += rouge_h['rouge1']
        score["rouge_2_human"] += rouge_h['rouge2']
        score["rouge_L_human"] += rouge_h['rougeL']


        meteor_s = meteor.compute(predictions=[x[gen_subject]],references=[x[sub]])
        meteor_h = meteor.compute(predictions=[x[gen_subject]] ,references=[[x[ann0],x[ann1],x[ann2]]])

        score["meteor_sub"] += meteor_s["meteor"]
        score["meteor_human"] += meteor_h["meteor"]

    print("The evalution scores are as follows:")

    for name,val in score.items():
        print(f"{name}: {val/len(data)}")

In [ ]:
metric(data_dev)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
100%|██████████| 1949/1949 [10:28<00:00,  3.10it/s]

The evalution scores are as follows:
rouge_1_sub: 0.18326589735748725
rouge_2_sub: 0.08425134431653598
rouge_L_sub: 0.17925176319352765
rouge_1_human: 0.2714969503080132
rouge_2_human: 0.140269422172909
rouge_L_human: 0.26688836519188697
meteor_sub: 0.15322635606321464
meteor_human: 0.23112576754471692


In [ ]:
metric(data_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
100%|██████████| 1901/1901 [10:00<00:00,  3.17it/s]

The evalution scores are as follows:
rouge_1_sub: 0.18193211866044764
rouge_2_sub: 0.08448332808947691
rouge_L_sub: 0.1794867368752446
rouge_1_human: 0.29092319096709124
rouge_2_human: 0.1536448987427922
rouge_L_human: 0.2855167298570702
meteor_sub: 0.14925497085477027
meteor_human: 0.2398384018615597


In [ ]:
import os
import csv
from tqdm import tqdm  # Import tqdm for progress bar


# Folder path containing the files
folder_path = "/content/AESLC/enron_subject_line/test"

# Output CSV file path
csv_output_path = "model_output.csv"

# Model saving path
model_save_path = "/content/model"
os.makedirs(model_save_path, exist_ok=True)

# Initialize an empty list to store data
data = []

# Number of iterations to save the model
save_model_interval = 5

# Iterate through files in the folder
for iteration, filename in enumerate(tqdm(os.listdir(folder_path), desc="Processing files")):
    if filename.endswith(".subject"):
        with open(os.path.join(folder_path, filename), 'r') as file:
          try:
            content = file.read()

            # Extracting prompt and annotations using string splitting
            prompt = content.split('@subject')[0].strip()[:800] + " @subject" if '@subject' in content else ''
            print(f"\nIteration: {iteration}")
            # print(f"Modified Prompt: {prompt}")

            subject = content.split('@subject')[1].split('@')[0].strip() if '@subject' in content else ''

            ann0 = content.split('@ann0')[1].split('@')[0].strip() if '@ann0' in content else ''
            # print(f"Annotation 0: {ann0}")

            ann1 = content.split('@ann1')[1].split('@')[0].strip() if '@ann1' in content else ''
            # print(f"Annotation 1: {ann1}")

            ann2 = content.split('@ann2')[1].split('@')[0].strip() if '@ann2' in content else ''
            # print(f"Annotation 2: {ann2}")

            # Generating a response (replace with your actual model inference logic)
            response = generate_response(my_model, my_tokenizer, prompt)
            # print(f"Generated Response: {response}")

            # Append data to the list
            data.append([prompt, subject, ann0, ann1, ann2, response])
          except Exception as e:
                print(f"Error reading file {filename}: {e}")

    # Save the model every 5 iterations
    # if iteration > 0 and iteration % save_model_interval == 0:
    #     model_save_filename = f"model_iteration_{iteration}.pth"
    #     model_save_filepath = os.path.join(model_save_path, model_save_filename)
    #     my_model.save_pretrained(model_save_filepath)
    #     print(f"Model saved at: {model_save_filepath}")

# Write data to CSV file
with open(csv_output_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header
    csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

    # Write data rows
    csvwriter.writerows(data)

print(f"CSV file '{csv_output_path}' created successfully.")


Processing files:   0%|          | 0/1906 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



Iteration: 0


Processing files:   0%|          | 1/1906 [00:07<3:58:01,  7.50s/it]


Iteration: 1


Processing files:   0%|          | 2/1906 [00:09<2:23:32,  4.52s/it]

...


Iteration: 1904


Processing files: 100%|█████████▉| 1905/1906 [39:31<00:01,  1.15s/it]


Iteration: 1905


Processing files: 100%|██████████| 1906/1906 [39:32<00:00,  1.24s/it]

CSV file 'model_output.csv' created successfully.


In [ ]:
data_test = pd.read_csv("/content/model_test_output.csv")

In [ ]:
# folder_path = "/content/sample_data/"

# # Output CSV file path
# csv_output_path = "model_output.csv"

# # Initialize an empty list to store data
# data = []

# # Iterate through files in the folder
# for filename in os.listdir(folder_path):
#     if filename.endswith(".subject"):
#         with open(os.path.join(folder_path, filename), 'r') as file:
#             content = file.read()

#             # Extracting prompt and annotations
#             prompt = content.split('@subject')[0].strip()
#             subject = content.split('@subject')[1].split('@')[0].strip()
#             ann0 = content.split('@ann0')[1].split('@')[0].strip()
#             ann1 = content.split('@ann1')[1].split('@')[0].strip()
#             ann2 = content.split('@ann2')[1].split('@')[0].strip()

#             # Generating a subject (replace with your actual model inference logic)
#             gen_subject = generate_response(my_model, my_tokenizer, prompt)

#             # Append data to the list
#             data.append([prompt, subject, ann0, ann1, ann2, gen_subject])

# # Write data to CSV file
# with open(csv_output_path, 'w', newline='') as csvfile:
#     csvwriter = csv.writer(csvfile)

#     # Write header
#     csvwriter.writerow(['Prompt', 'Subject', 'Annotation 0', 'Annotation 1', 'Annotation 2', 'Generated Subject'])

#     # Write data rows
#     csvwriter.writerows(data)

# print(f"CSV file '{csv_output_path}' created successfully.")

In [ ]:
# dataset = TextDataset(tokenized_data, tokenizer=tokenizer, block_size=128)
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# # Create DataLoader
# train_dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
# # Step 3: Format data for training
# class EmailDataset(Dataset):
#     def __init__(self, tokenized_data):
#         self.tokenized_data = tokenized_data

#     def __len__(self):
#         return len(self.tokenized_data)

#     def __getitem__(self, idx):
#         return self.tokenized_data[idx]

# dataset = EmailDataset(tokenized_data)
# dataloader = DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
# # Step 4: Fine-tune the model
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# optimizer = AdamW(model.parameters(), lr=5e-5)



In [ ]:
# # Training loop for 2 epochs
# num_epochs = 1
# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     for batch in dataloader:
#         input_ids = batch['input_ids']
#         labels = batch['labels']

#         outputs = model(input_ids, labels=labels)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         optimizer.zero_grad()

#         print(f"Batch loss: {loss.item()}")